In [3]:
import os
import requests
import random
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm

In [2]:
def scrape_city(city, state = "CA"):
    """
    This function searches for information about recently sold houses in the chosen city on realtor.com
    Individual pages for each house are saved locally as html files in the folder named after the city of interest
    """
    
    # creating folder for the city data if it does not already exist
    if not os.path.exists(city):
        os.makedirs(city)
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36', 
               'Accept-Language': 'en-US, en;q=0.5'}
    session = requests.Session()
    
    url = "https://www.realtor.com/realestateandhomes-search/{}_{}/show-recently-sold".format(city, state)
    search_page = session.get(url, headers = headers)
    search_page = BeautifulSoup(search_page.content, 'html.parser')
    sleep(random.randint(5, 10))
    
    # extracting the number of pages
    count_results = search_page.find_all('span', {"data-testid": "results-header-count"})[0].text
    count_results = count_results.split(" ")[0]
    count_results = int(count_results.replace(",", ""))
    num_pages = int(count_results / 42) + 1
    
    k = 0
    for page_num in range(1, num_pages + 1):
        # assessing search pages one by one
        url_page = url + "/pg-{}".format(str(page_num))
        search_page = session.get(url_page, headers = headers)
        sleep(random.randint(5, 10))
        search_page = BeautifulSoup(search_page.content, 'html.parser')
        homes = search_page.find_all('a', {"data-testid": "property-anchor"})
        
        i = 0
        while i < len(homes):
            # extracting links to individual house pages from the search page
            home_url = homes[i]['href']
            # requesting individual house pages and saving locally them as html files
            home_page = session.get("https://www.realtor.com/" + home_url, headers = headers)
            with open("{}/home_page_{}.html".format(city, str(k)), 'wb') as f:
                f.write(home_page.content)
            sleep(random.randint(5, 10))
            i += 1
            k += 1

In [4]:
def scrape_home_pages(folder, output_file_name):
    """
    This function scrapes the data from individual house pages downloaded from realtor.com
    The data is then aggregated as a table and is saved as a csv file
    
    Parameters:
    folder (str): path to the folder with downloaded house pages
    output_file_name(str): path to save the resulting csv file
    
    Return:
    pd.DataFrame containing aggregated housing information
    """
    
    data = {"sold_date": [],
            "bedrooms": [],
            "bathrooms": [],
            "build_year": [],
            "build_type": [],
            "area": [],
            "lot_area": [],
            "parking_spots": [],
            "homeowners_association": [],
            "zip_code": [], 
            "nearby_elem_school": [],
            "nearby_middle_school": [],
            "nearby_high_school": [],
            "nh_median_price": [],
            "nh_days_on_market": [],
            "nh_price_per_sqft": [],
            "selling_broker": [],
            "buying_broker": [],
            "price": []}
    
    for filename in tqdm(os.listdir(folder)):
        try:
            file = open("{}/{}".format(folder, filename), "r", encoding = 'utf8')
            home = BeautifulSoup(file.read(), 'html.parser')

            # extracting sold date
            sold_date = home.find_all('span', {"data-label": "property-meta-sold-date"})[0].text[8:]
            sold_date = sold_date[: len(sold_date) - 1]
            sold_date = datetime.strptime(sold_date, '%B %d, %Y')

            # bedrooms
            beds = home.find_all('li', {"data-label": "property-meta-beds"})[0]
            beds = beds.find_all('span', {"class": "data-value"})[0].text
            beds = int(beds)

            # bathrooms
            baths = home.find_all('li', {"data-label": "property-meta-bath"})[0]
            baths = baths.find_all('span', {"class": "data-value"})[0].text
            baths = int(baths)

            def search_public_records(public_records, keyterm, to_drop = []):
                output = public_records[public_records.index(keyterm) + len(keyterm): ]
                end_pos = output.find("\n")
                if end_pos > -1:
                    output = output[: end_pos]
                for elem in to_drop:
                    output = output.replace(elem, "")
                return output

            public_records = home.find_all('div', {"id": "ldp-detail-public-records"})[0].text

            # year built
            build_year = search_public_records(public_records, "Year built: ")
            build_year = int(build_year)

            # building type
            build_type = search_public_records(public_records, "Property type: ")

            # area
            area = search_public_records(public_records, "House size: ", to_drop = [",", " sq ft"])
            area = int(area)

            # lot_area
            lot_area = search_public_records(public_records, "Lot size: ")
            lot_area = int(lot_area)

            # parking
            try:
                parking = home.text[home.text.index("Parking Total: ") + len("Parking Total: "): ]
                parking = int(parking.split(" ")[0])
            except:
                parking = 0

            # homeowners association
            try:
                ha = home.text[home.text.index("Association: ") + len("Association: "): ]
                ha = str(ha.split(" ")[0])
                if ha.startswith("No"):
                    ha = False
                else:
                    ha = True
            except:
                ha = False

            # zip code
            zip_code = home.find_all('span', {"itemprop": "postalCode"})[0].text

            # distance to the nearest schools
            schools = home.find_all('table', {"class": "table table-clear table-heading-unstyled table-school school-rating-lg"})[0].text.split("\n")
            schools = [s for s in schools if " mi" in s]
            schools = schools[0:3]
            schools = [float(s.split(" ")[0]) for s in schools]
            nearby_elem_school, nearby_middle_school, nearby_high_school = schools

            # neighborhood
            nh = home.find_all('div', {"class": "neighborhood-flex-item"})

            # median sales price in the neighborhood
            nh_median_price = nh[1].text
            nh_median_price = nh_median_price[nh_median_price.index("$") + 1: nh_median_price.index("Median Sales Price")]
            nh_median_price = int(nh_median_price.replace(",", ""))

            # median days on the market for the neighborhood
            nh_days_on_market = nh[2].text
            nh_days_on_market = nh_days_on_market[: nh_days_on_market.index("Median Days on Market")]
            nh_days_on_market = int(nh_days_on_market)

            # median price per sq ft in the neighborhood
            nh_price_per_sqft = nh[3].text
            nh_price_per_sqft = nh_price_per_sqft[nh_price_per_sqft.index("$") + 1: nh_price_per_sqft.index("Price Per Sq Ft")]
            nh_price_per_sqft = int(nh_price_per_sqft.replace(",", ""))

            # selling broker
            selling_broker = home.find_all('span', {"data-label": "branding-seller-broker-name"})[0].text[1:]

            # buying broker
            buying_broker = home.find_all('span', {"data-label": "branding-buyer-broker-name"})[0].text[1:]

            # price
            price = home.find_all('div', {"class": "display-inline"})[1].text
            price = price[price.index("$") + 1: ]
            price = price[: price.index(" ")]
            price = price.replace(",", "")
            price = int(price)

            # aggregating the data for individual homes
            data["sold_date"].append(sold_date)
            data["bedrooms"].append(beds)
            data["bathrooms"].append(baths)
            data["build_year"].append(build_year)
            data["build_type"].append(build_type)
            data["area"].append(area)
            data["lot_area"].append(lot_area)
            data["parking_spots"].append(parking)
            data["homeowners_association"].append(ha)
            data["zip_code"].append(zip_code)
            data["nearby_elem_school"].append(nearby_elem_school)
            data["nearby_middle_school"].append(nearby_middle_school)
            data["nearby_high_school"].append(nearby_high_school)
            data["nh_median_price"].append(nh_median_price)
            data["nh_days_on_market"].append(nh_days_on_market)
            data["nh_price_per_sqft"].append(nh_price_per_sqft)
            data["selling_broker"].append(selling_broker)
            data["buying_broker"].append(buying_broker)
            data["price"].append(price)
        except:
            continue
        
    df = pd.DataFrame(data)
    df.to_csv(output_file_name, index = False)
    return df

In [ ]:
scrape_city("San-Francisco")

In [5]:
df = scrape_home_pages("San-Francisco", "sf_sold_houses.csv")

  0%|          | 0/4023 [00:00<?, ?it/s]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1899 entries, 0 to 1898
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   sold_date               1899 non-null   object 
 1   bedrooms                1899 non-null   int64  
 2   bathrooms               1899 non-null   int64  
 3   build_year              1899 non-null   int64  
 4   build_type              1899 non-null   object 
 5   area                    1899 non-null   int64  
 6   lot_area                1899 non-null   int64  
 7   parking_spots           1899 non-null   int64  
 8   homeowners_association  1899 non-null   bool   
 9   zip_code                1899 non-null   int64  
 10  nearby_elem_school      1899 non-null   float64
 11  nearby_middle_school    1899 non-null   float64
 12  nearby_high_school      1899 non-null   float64
 13  nh_median_price         1899 non-null   int64  
 14  nh_days_on_market       1899 non-null   

In [10]:
# creating mongodb collection

import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = myclient["realtor_data"]

mycol = mydb["SF_houses"]

mycol.insert_one({"index":"sold_date","data":df.to_dict("records")})

myclient.close()